In [2]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [112]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
URL= "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"

spacex_df = pd.read_csv(URL)
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = dash.Dash(__name__)



# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),

                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)

                                # H2 为2级别标签，可换成其他。Height参数：标签下方空开距离. Margin right参数：CSS参数，右边距为当前字体大小的2倍, placeholder 为菜单本身的值
                                # dropdown这里的下拉菜单可以简便使用 df.['A'].unique()
                                
                                html.Div([html.H2("Select Site", style={'height':'20px','font-size':35,'margin-right':'2em'}), 
                                          dcc.Dropdown(id='site-dropdown',options=[{'label':"All Sites", 'value':'ALL'},
                                                              {'label':"CCAFS LC-40",'value':'CCAFS LC-40'},
                                                              {'label':"VAFB SLC-4E", 'value':"VAFB SLC-4E"},
                                                              {'label':"KSC LC-39A", 'value':"KSC LC-39A"},
                                                              {"label":"CCAFS SLC-40",'value':"CCAFS SLC-40"}],
                                                       searchable=True,value = 'ALL', placeholder='Select a Launch Site here')]),
                                          
                                                                                                                               
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),


                                #html.P() 创立段落paragraph
                                # css 'font-weight' 为bold粗体字
                                
                                

                                # TASK 3: Add a slider to select payload range
                                html.P("Payload range (Kg):", style={'font-size':15,'font-weight': 'bold'}),

                                #step表示滑动一下，间隔距离多少
                                #用marks在固定位置手动添加标签，
                                dcc.RangeSlider(id='payload-slider', min=0, max=10000, step=1000,marks={0:'0', 2000:'2000',4000:"4000",6000:"6000",8000:"8000",10000:"10000"},
                                value=[min_payload, max_payload]),
                                html.Br(),
                                

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output

# callback decorator
# figure为图表组件
@app.callback(Output(component_id='success-pie-chart', component_property="figure"), Input(component_id="site-dropdown", component_property="value"))

# callback function
def get_pie_chart(entered_site):

    if entered_site =="ALL":
        filtered_df =spacex_df[spacex_df['class']==1]
        fig = px.pie(filtered_df, values='class',
                    names='Launch Site',
                    title = 'Total Success Launches')
        return fig
    else:
        filtered_df=spacex_df[spacex_df['Launch Site']==entered_site]
        fig = px.pie(filtered_df, names='class', title = f'Success Launch by {entered_site}')

        return fig


@app.callback(Output(component_id='success-payload-scatter-chart', component_property="figure"), [Input(component_id="site-dropdown", component_property="value"),Input(component_id ='payload-slider', component_property='value')])
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
def payload(site_dropdown_value, payload_slider_value):
    if site_dropdown_value=='ALL':
        filtered_df=spacex_df[(spacex_df['Payload Mass (kg)']>payload_slider_value[0])&(spacex_df['Payload Mass (kg)']<payload_slider_value[1])]
        fig = px.scatter(filtered_df, x='Payload Mass (kg)', y='class',title = 'distribution of Payload Mass by Class', color='Booster Version Category')
        return fig
    else:
        filtered_df = spacex_df[spacex_df['Launch Site']==site_dropdown_value]
        filtered_df=filtered_df[(filtered_df['Payload Mass (kg)']>payload_slider_value[0])&(filtered_df['Payload Mass (kg)']<payload_slider_value[1])]
        fig =px.scatter(filtered_df, x='Payload Mass (kg)',y='class', title = f'distribution of Payload Mass by {site_dropdown_value}', color='Booster Version Category')
        return fig




# Run the app
if __name__ == '__main__':
    app.run_server(debug= True, port=8051)
    


In [92]:
spacex_df

,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
2,2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0
8,8,10,CCAFS LC-40,0,1316.0,F9 v1.1,v1.1
13,13,15,CCAFS LC-40,0,570.0,F9 v1.1 B1013,v1.1
26,26,6,VAFB SLC-4E,0,500.0,F9 v1.1 B1003,v1.1
27,27,21,VAFB SLC-4E,0,553.0,F9 v1.1 B1017,v1.1
30,30,40,VAFB SLC-4E,1,475.0,F9 FT B1038.1,FT
54,54,53,CCAFS SLC-40,1,362.0,F9 B4 B1045.1,B4
